In [27]:
import pandas as pd 
import requests
import os
import urllib.request

In [28]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Extracao_Dados_Operadoras'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [29]:
CAMINHO_ARQUIVOS = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_download']

In [30]:
NOME_ARQUIVO_DOWNLOAD = CAMINHO_ARQUIVOS+"caracteristicas_produtos_saude_suplementar.csv"
NOME_ARQUIVO_FINAL = CAMINHO_ARQUIVOS+PARAMETROS['nome_arquivo_final']

In [31]:
PARAMETROS

{'caminho_arquivo_download': 'DADOS_OPERADORAS/',
 'url_csv': 'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/caracteristicas_produtos_saude_suplementar/caracteristicas_produtos_saude_suplementar.csv',
 'nome_arquivo_final': 'dados_operadoras.csv',
 'url_csv_operadoras_ativas': 'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/Relatorio_cadop.csv'}

In [32]:
PARAMETROS_GERAIS

{'pasta_raiz_dados': '/opt/DADOS_ANS_RESUMIDO/'}

In [33]:
if (not(os.path.exists(CAMINHO_ARQUIVOS))):
    os.mkdir(CAMINHO_ARQUIVOS)

In [34]:
urllib.request.urlretrieve(PARAMETROS['url_csv'], NOME_ARQUIVO_DOWNLOAD)

('/opt/DADOS_ANS_RESUMIDO/DADOS_OPERADORAS/caracteristicas_produtos_saude_suplementar.csv',
 <http.client.HTTPMessage at 0x7fbe60b55290>)

In [35]:
df = pd.read_csv(NOME_ARQUIVO_DOWNLOAD,sep=";",encoding="ISO-8859-1",dtype=str)

In [36]:
df.head()

,#id_plano,Cd_Plano,Nome_Plano,Registro_ANS,Razao_Social,Nome_Fantasia,Codigo_Modalidade,Porte_Operadora,Vigencia_Plano,Codigo_Contratacao,...,Grupo_Segmentacao_Assistencial,Cobertura,Tipo_Financiamento,Codigo_Abrangencia_Cobertura,Fator_Moderador,Log_Fator_Moderador,Acomodacao_Hospitalar,ID_Situacao_Plano,Data_Situacao_Plano,Data_Registro_Plano
0,1335931,401404989,EXECUTIVO II,000477,SOMPO SAÚDE SEGUROS S.A.,NaN,28,MÉDIA,P,2,...,AMB + HOSP C/ OBST,MÉDICO-HOSPITALAR,Pre-estabelecido,4,CO-PARTICIPACÃO,1,INDIVIDUAL,ATIVO,30/12/1998,30/12/1998
1,1341425,405906999,Plano A-Ambulatorial Hospitalar Com Obstetríci...,311944,UNIMED DE ADAMANTINA-COOPERATIVA DE TRABALHO M...,UNIMED DE ADAMANTINA,22,PEQUENA,P,2,...,AMB + HOSP C/ OBST,MÉDICO-HOSPITALAR,Pre-estabelecido,4,CO-PARTICIPACÃO,1,COLETIVA,ATIVO,28/01/1999,28/01/1999
2,1342396,436359011,Senior Standard 751,319180,SÃO DOMINGOS SAÚDE- ASSISTÊNCIA MÉDICA LTDA,SÃO DOMINGOS SAÚDE,25,PEQUENA,P,1,...,AMB + HOSP S/ OBST,MÉDICO-HOSPITALAR,Pre-estabelecido,4,AUSENTE,0,COLETIVA,ATIVO COM COMERCIALIZAÇÃO SUSPENSA,26/10/2015 10:15:15,07/08/2001
3,1342410,440002020,Executivo 742,319180,SÃO DOMINGOS SAÚDE- ASSISTÊNCIA MÉDICA LTDA,SÃO DOMINGOS SAÚDE,25,PEQUENA,P,1,...,AMB + HOSP C/ OBST,MÉDICO-HOSPITALAR,Pre-estabelecido,5,CO-PARTICIPACÃO,1,INDIVIDUAL,ATIVO COM COMERCIALIZAÇÃO SUSPENSA,02/02/2009 09:57:03,12/11/2002
4,1342450,400630985,VITA - B (Enfermaria),316296,MASSA FALIDA DE VITA SAÚDE ADMINISTRAÇÃO HOSPI...,CASA DE SAÚDE SÃO SEBASTIÃO,25,SEM BENEFICIÁRIOS,P,1,...,AMB + HOSP C/ OBST,MÉDICO-HOSPITALAR,Pre-estabelecido,4,AUSENTE,0,COLETIVA,CANCELADO,29/10/2012 13:35:45,29/12/1998


In [37]:
df_dados_operadora = df[["Registro_ANS","Razao_Social","Nome_Fantasia","Codigo_Modalidade","Porte_Operadora"]]

In [38]:
df_dados_operadora.drop_duplicates(inplace=True)

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_dados_operadora['Nome_Fantasia'] = df_dados_operadora['Nome_Fantasia'].fillna(df_dados_operadora['Razao_Social'])

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_cooperativas = pd.read_csv('Cooperativas Médicas Ativas_CADOP.txt',sep="\t",dtype=str,encoding="ISO-8859-1")

In [41]:
lista_cooperativas = df_cooperativas["Registro_ANS"].tolist()


In [42]:
df_dados_operadora = df_dados_operadora[df_dados_operadora["Registro_ANS"].isin(lista_cooperativas)]

In [43]:
df_dados_operadora.to_csv(NOME_ARQUIVO_FINAL,index=False)

In [44]:
NOME_ARQUIVO_FINAL

'/opt/DADOS_ANS_RESUMIDO/DADOS_OPERADORAS/dados_operadoras.csv'